In [2]:
# Import required libraries
import numpy as np
import pandas as pd

### Sample Solution provided

In [3]:
nInst = 50
currentPos = np.zeros(nInst)


def getMyPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape
    if (nt < 2):
        return np.zeros(nins)
    lastRet = np.log(prcSoFar[:, -1] / prcSoFar[:, -2])
    lNorm = np.sqrt(lastRet.dot(lastRet))
    lastRet /= lNorm
    rpos = np.array([int(x) for x in 5000 * lastRet / prcSoFar[:, -1]])
    currentPos = np.array([int(x) for x in currentPos+rpos])
    return currentPos

### Evaluation of sample

In [4]:
nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000


def loadPrices(fn):
    global nt, nInst
    df = pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    (nt, nInst) = df.shape
    return (df.values).T


pricesFile = "./prices.txt"
prcAll = loadPrices(pricesFile)
print("Loaded %d instruments for %d days" % (nInst, nt))


def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_, nt) = prcHist.shape
    for t in range(250, 501):
        prcHistSoFar = prcHist[:, :t]
        newPosOrig = getMyPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:, -1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        newPos = np.clip(newPosOrig, -posLimits, posLimits)
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        print("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" %
              (t, value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu, plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)


(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd
print("=====")
print("mean(PL): %.1lf" % meanpl)
print("return: %.5lf" % ret)
print("StdDev(PL): %.2lf" % plstd)
print("annSharpe(PL): %.2lf " % sharpe)
print("totDvolume: %.0lf " % dvol)
print("Score: %.2lf" % score)


Loaded 50 instruments for 500 days
Day 250 value: -23.79 todayPL: $-23.79 $-traded: 23790 return: -0.00100
Day 251 value: -86.08 todayPL: $-62.29 $-traded: 48793 return: -0.00176
Day 252 value: -109.18 todayPL: $-23.10 $-traded: 72720 return: -0.00150
Day 253 value: -123.23 todayPL: $-14.05 $-traded: 97808 return: -0.00126
Day 254 value: -120.46 todayPL: $2.76 $-traded: 122814 return: -0.00098
Day 255 value: -14.49 todayPL: $105.97 $-traded: 143803 return: -0.00010
Day 256 value: -19.16 todayPL: $-4.66 $-traded: 166776 return: -0.00011
Day 257 value: -80.93 todayPL: $-61.77 $-traded: 189951 return: -0.00043
Day 258 value: -171.97 todayPL: $-91.04 $-traded: 215583 return: -0.00080
Day 259 value: -123.18 todayPL: $48.79 $-traded: 238044 return: -0.00052
Day 260 value: -271.54 todayPL: $-148.35 $-traded: 261668 return: -0.00104
Day 261 value: -222.76 todayPL: $48.78 $-traded: 284770 return: -0.00078
Day 262 value: -133.62 todayPL: $89.14 $-traded: 306173 return: -0.00044
Day 263 value: -2